In [2]:
!pip install psycopg2

     ---------------------------------------- 1.2/1.2 MB 1.4 MB/s eta 0:00:00


In [116]:
!pip install textblob

  Using cached textblob-0.17.1-py2.py3-none-any.whl (636 kB)


In [119]:
import psycopg2
import pandas as pd
import re
from textblob import TextBlob

In [58]:
conn = psycopg2.connect(
host="localhost",
port="5432",
database="Bookscrape",
user="postgres",
password="#Tolexy5038"
)

cur = conn.cursor()

In [59]:
data = "SELECT * FROM pfizer"

cur.execute(data)


In [60]:
rows = cur.fetchall()

time = []
tweets = []

for row in rows:
    time.append(row[0])
    tweets.append(row[1])
    
conn.commit()
cur.close()

In [91]:
pfizer_dict = {'time': time, 'tweets': tweets}

In [97]:
df = pd.DataFrame(pfizer_dict)

In [98]:
df

,time,tweets
0,2023-05-22 21:28:35+01:00,@MajesticTrout @annamerlan @motherboard No. Pf...
1,2023-05-22 21:28:32+01:00,RT @sonia_elijah: My discussion with @BLNewsMe...
2,2023-05-22 21:28:22+01:00,RT @DrewSav: This is literally Ron DeSantis wa...
3,2023-05-22 21:28:22+01:00,RT @jathorpmfm: This is absolutely an abominat...
4,2023-05-22 21:28:10+01:00,@DidierMaisto Le staff de Pfizer est vacciné ?
...,...,...
176,2023-05-22 22:18:58+01:00,RT @DrewSav: This is literally Ron DeSantis wa...
177,2023-05-22 22:18:58+01:00,RT @EpochTimes: “They made him think that it w...
178,2023-05-22 22:18:49+01:00,RT @voodoobeans_: The vaccine injured &amp; be...
179,2023-05-22 22:18:32+01:00,RT @jathorpmfm: This is absolutely an abominat...


In [111]:
def clean_tweet(text):
    text = text.lower()
    text = re.sub(r"@\w+","",text) #removes the  twitter mentions
    text = re.sub(r"@\w+:","",text)
    text = re.sub(r"#\w+","",text) #removes the tags
    text = re.sub(r"http\S+", "", text) #removes the web link
    text = re.sub(r"wwww\S+","",text)
    text = re.sub(r"RT\s+", "", text)
    text = re.sub(r"rt\s+", "", text)
    text = re.sub('[":"]', "", text)
    text = re.sub(r'"', '', text)
    
    return text

In [112]:
df['tweets'] = df['tweets'].apply(clean_tweet)

In [113]:
df

,time,tweets
0,2023-05-22 21:28:35+01:00,no. pfizer’s covid-19 vaccine did.
1,2023-05-22 21:28:32+01:00,my discussion with on the devastating harms ...
2,2023-05-22 21:28:22+01:00,this is literally ron desantis watching the u...
3,2023-05-22 21:28:22+01:00,this is absolutely an abomination of science....
4,2023-05-22 21:28:10+01:00,le staff de pfizer est vacciné ?
...,...,...
176,2023-05-22 22:18:58+01:00,this is literally ron desantis watching the u...
177,2023-05-22 22:18:58+01:00,“they made him think that it was a true vacci...
178,2023-05-22 22:18:49+01:00,the vaccine injured &amp; bereaved rally and ...
179,2023-05-22 22:18:32+01:00,this is absolutely an abomination of science....


In [120]:
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity
 
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

df['subjectivity'] = df['tweets'].apply(getSubjectivity)
df['polarity'] = df['tweets'].apply(getPolarity)

In [121]:
df

,time,tweets,subjectivity,polarity
0,2023-05-22 21:28:35+01:00,no. pfizer’s covid-19 vaccine did.,0.000000,0.000
1,2023-05-22 21:28:32+01:00,my discussion with on the devastating harms ...,1.000000,-1.000
2,2023-05-22 21:28:22+01:00,this is literally ron desantis watching the u...,0.333333,0.250
3,2023-05-22 21:28:22+01:00,this is absolutely an abomination of science....,0.450000,0.100
4,2023-05-22 21:28:10+01:00,le staff de pfizer est vacciné ?,0.000000,0.000
...,...,...,...,...
176,2023-05-22 22:18:58+01:00,this is literally ron desantis watching the u...,0.333333,0.250
177,2023-05-22 22:18:58+01:00,“they made him think that it was a true vacci...,0.325000,0.175
178,2023-05-22 22:18:49+01:00,the vaccine injured &amp; bereaved rally and ...,0.000000,0.000
179,2023-05-22 22:18:32+01:00,this is absolutely an abomination of science....,0.450000,0.100
